# Economics 600a Fall 2025 Prof. P. Haile Homework Assignment 1

## 1 Overview
You will estimate demand and supply in a stylized model of the market for pay-TV services. You will use a matrix programming language of your choice to create your own fake data set for the industry and do some relatively simple estimation. Then, using the **pyBLP** package of Conlon and Gortmaker, you will estimate the model and perform some merger simulations.

The pyBLP package has excellent documentation and a very helpful tutorial (which covers merger simulation), both easy to find via Google.

Please submit (on canvas) a single PDF document presenting your answers to the questions below, requested results, and well documented code. Write this up nicely, with properly formatted tables and discussion of results. You may work in groups on the coding. However, your write-ups should be your own work, and you must describe all collaboration at the beginning of your submission; this includes any use of AI.

## 2 Model
There are $T$ markets, each with four inside goods $j \in \{1,2,3,4\}$ and an outside option. Goods 1 and 2 are satellite television services (e.g., DirecTV and Dish); goods 3 and 4 are wired television services (e.g., Frontier and Comcast in New Haven). The conditional indirect utility of consumer $i$ for good $j$ in market $t$ is given by

\begin{align*}
u_{ijt} &= \beta^{(1)} x_{jt} + \beta_i^{(2)} satellite_{jt} + \beta_i^{(3)} wired_{jt} + \alpha p_{jt} + \xi_{jt} + \epsilon_{ijt} \quad j > 0 \\
u_{i0t} &= \epsilon_{i0t},
\end{align*}

where $x_{jt}$ is a measure of good $j$'s quality, $p_{jt}$ is its price, $satellite_{jt}$ is an indicator equal to 1 for the two satellite services, and $wired_{jt}$ is an indicator equal to 1 for the two wired services. The remaining notation is as usual in the class notes, including the i.i.d. type-1 extreme value $\epsilon_{ijt}$. Each consumer purchases the good giving them the highest conditional indirect utility.

Goods are produced by single-product firms. Firm $j$'s (log) marginal cost in market $t$ is

\begin{equation*}
\ln mc_{jt} = \gamma^{(0)} + w_{jt} \gamma^{(1)} + \omega_{jt}/8,
\end{equation*}

where $w_{jt}$ is an observed cost shifter. Firms compete by simultaneously choosing prices in each market under complete information. Firm $j$ has profit

\begin{equation*}
\pi_{jt} = \max_{p_{jt}} (p_{jt} - mc_{jt}) s_{jt}(p_t).
\end{equation*}

## 3 Generate Fake Data

Generate a data set from the model above. Let

\begin{align*}
\beta^{(1)} &= 1, \quad \beta_i^{(k)} \sim \text{iid } N(4,1) \text{ for } k=2,3 \\
\alpha &= -2 \\
\gamma^{(0)} &= 1/2, \quad \gamma^{(1)} = 1/4.
\end{align*}

In [2]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
from scipy.special import logsumexp
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import time
import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

### 1. 
Draw the exogenous product characteristic $x_{jt}$ for $T=600$ geographically defined markets (e.g., cities). Assume each $x_{jt}$ is equal to the absolute value of an iid standard normal draw, as is each $w_{jt}$. Simulate demand and cost unobservables as well, specifying

\begin{equation*}
\left(
\begin{array}{c}
\xi_{jt} \\
\omega_{jt}
\end{array}
\right) \sim N\left( \left(
\begin{array}{c}
0 \\
0
\end{array}
\right), \left(
\begin{array}{cc}
1 & 0.25 \\
0.25 & 1
\end{array}
\right) \right) \quad \text{iid across } j,t.
\end{equation*}

In [3]:
np.random.seed(1995)

# Model parameters
T, J = 600, 4
alpha, beta1 = -2, 1
beta2, beta3 = 4, 4  
sigma_satellite, sigma_wired = 1, 1
gamma0, gamma1 = 0.5, 0.25

# Product data structure
data = [{'market_id': t, 'firm_id': j+1, 'product_id': j} for t in range(T) for j in range(J)]
product_data = pd.DataFrame(data)

# Exogenous variables: x_jt and w_jt as absolute values of iid standard normal draws
product_data['x'] = np.abs(np.random.normal(0, 1, len(product_data)))
product_data['w'] = np.abs(np.random.normal(0, 1, len(product_data)))

# Indicators
product_data['satellite'] = product_data['firm_id'].isin([1, 2]).astype(int)
product_data['wired'] = product_data['firm_id'].isin([3, 4]).astype(int)

# Add BLP-style instruments
product_data['sum_x_competitors'] = product_data.groupby('market_id')['x'].transform('sum') - product_data['x']
product_data['sum_w_competitors'] = product_data.groupby('market_id')['w'].transform('sum') - product_data['w']

# Unobservables: ξ_jt and ω_jt with covariance matrix [[1, 0.25], [0.25, 1]]
cov_matrix = np.array([[1, 0.25], [0.25, 1]])
A = np.linalg.cholesky(cov_matrix)
z = np.random.normal(0, 1, (len(product_data), 2))
unobs = z @ A.T
product_data['xi'] = unobs[:, 0]  # demand unobservable
product_data['omega'] = unobs[:, 1]  # cost unobservable

print("Question 1 completed:")
print(f"Generated {len(product_data)} observations across {T} markets")
print(f'x range: {product_data['x'].min():.3f} to {product_data['x'].max():.3f}')
print(f"w range: {product_data["w"].min():.3f} to {product_data["w"].max():.3f}")
print(f"ξ-ω correlation: {product_data[['xi', 'omega']].corr().iloc[0,1]:.3f} (target: 0.25)")
print(f"Satellite products: {product_data["satellite"].sum()}, Wired products: {product_data["wired"].sum()}")

Question 1 completed:
Generated 2400 observations across 600 markets
x range: 0.001 to 3.534
w range: 0.000 to 3.621
ξ-ω correlation: 0.240 (target: 0.25)
Satellite products: 1200, Wired products: 1200


### 2. Solve for the equilibrium prices for each good in each market.

**(a)** Start by writing a procedure to approximate the derivatives of market shares with respect to prices (taking prices, shares, x, and demand parameters as inputs). The key steps are:

(i) For each $jt$, write the choice probability for good $j$, $s_{jt}$, as a weighted average (integral) of the (multinomial logit) choice probabilities conditional on the value of each consumer's random coefficients;

The market share for good $j$ in market $t$, $s_{jt}$, is the probability that a consumer chooses good $j$:

$$s_{jt} = \int P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)}) f(\beta_i^{(2)}, \beta_i^{(3)}) d\beta_i^{(2)} d\beta_i^{(3)}$$

where $P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)})$ is the multinomial logit choice probability conditional on the random coefficients.

Given the random coefficients $\beta_i^{(2)}$ and $\beta_i^{(3)}$ (with means $\beta^{(2)} = 4$, $\beta^{(3)} = 4$ and variances $\sigma_2^2 = 1$, $\sigma_3^2 = 1$), the conditional utility becomes:

$$u_{ijt} = \beta^{(1)} x_{jt} + \beta_i^{(2)} satellite_{jt} + \beta_i^{(3)} wired_{jt} + \alpha p_{jt} + \xi_{jt} + \epsilon_{ijt}$$

Since $\epsilon_{ijt}$ are i.i.d. Type-1 extreme value, the conditional choice probability follows the multinomial logit form:

$$P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)}) = \frac{\exp(\delta_{jt} + \mu_{jt}^i)}{\sum_{k=1}^J \exp(\delta_{kt} + \mu_{kt}^i) + 1}$$

where:
- $\delta_{jt} = \beta^{(1)} x_{jt} + \alpha p_{jt} + \xi_{jt}$ (mean utility component)
- $\mu_{jt}^i = \beta_i^{(2)} satellite_{jt} + \beta_i^{(3)} wired_{jt}$ (random utility component)

**Final Expression:**

$$s_{jt} = \int \frac{\exp(\delta_{jt} + \beta_i^{(2)} satellite_{jt} + \beta_i^{(3)} wired_{jt})}{\sum_{k=1}^J \exp(\delta_{kt} + \beta_i^{(2)} satellite_{kt} + \beta_i^{(3)} wired_{kt}) + 1} \phi(\beta_i^{(2)}, \beta_i^{(3)}) d\beta_i^{(2)} d\beta_i^{(3)}$$

where $\phi(\cdot, \cdot)$ is the bivariate normal density with mean $(\beta^{(2)}, \beta^{(3)}) = (4, 4)$ and covariance matrix $\text{diag}(1, 1)$.

This integral is approximated in the code using Monte Carlo simulation with draws from the normal distribution of $(\beta_i^{(2)}, \beta_i^{(3)})$.

(ii) Anticipating differentiation under the integral sign, derive the analytical expression for the derivative of the integrand with respect to each $p_{kt}$.

The integrand is the conditional choice probability $P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)})$, which depends on prices through the mean utility component $\delta_{jt} = \beta^{(1)} x_{jt} + \alpha p_{jt} + \xi_{jt}$.

Since $p_{kt}$ appears in $\delta_{kt}$, the derivative with respect to $p_{kt}$ affects the choice probability.

For the multinomial logit model, the derivative of the choice probability with respect to a price is:

$$\frac{\partial P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)})}{\partial p_{kt}} = \alpha P(j|\beta_i) \left( \delta_{jk} - P(k|\beta_i) \right)$$

where $\delta_{jk} = 1$ if $j = k$ and 0 otherwise.

This follows from the general formula for multinomial logit derivatives:

$$\frac{\partial P_j}{\partial p_k} = \alpha P_j (I_{jk} - P_k)$$

where $I_{jk}$ is the indicator function equal to 1 if $j = k$.

Therefore, the derivative of the integrand (conditional choice probability) with respect to $p_{kt}$ is:

$$\frac{\partial}{\partial p_{kt}} \left[ \frac{\exp(\delta_{jt} + \mu_{jt}^i)}{\sum_{m=1}^J \exp(\delta_{mt} + \mu_{mt}^i) + 1} \right] = \alpha \cdot \frac{\exp(\delta_{jt} + \mu_{jt}^i)}{\sum_{m=1}^J \exp(\delta_{mt} + \mu_{mt}^i) + 1} \left( I_{jk} - \frac{\exp(\delta_{kt} + \mu_{kt}^i)}{\sum_{m=1}^J \exp(\delta_{mt} + \mu_{mt}^i) + 1} \right)$$

3. Use the expression you obtained in (2) and simulation draws of the random coefficients to approximate the integral that corresponds to $\partial s_{jt}/\partial p_{kt}$ for each $j$ and $k$ (i.e., replace the integral with the mean over the values at each simulation draw). Recall the advice in the lecture regarding "jittering."

In [ ]:
def compute_market_shares(prices, market_data, v_draws=None, n_draws=1000, safe=True):
    """Compute market shares using simulation"""
    if v_draws is None:
        v_draws = np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws)
    J = len(market_data)
    n_draws = v_draws.shape[0]
    x, xi = market_data['x'].values, market_data['xi'].values
    sat, wired = market_data['satellite'].values, market_data['wired'].values
    utilities = (beta1 * x + xi + v_draws[:, 0:1] * sat + v_draws[:, 1:2] * wired + alpha * prices)
    utilities = np.column_stack([utilities, np.zeros(n_draws)])
    if safe:
        utility_reduction = np.clip(utilities.max(axis=0, keepdims=True), 0, None)
        utilities -= utility_reduction
        exp_u = np.exp(utilities)
        scale = np.exp(-utility_reduction)
        choice_probs = exp_u / (scale + exp_u.sum(axis=1, keepdims=True))
    else:
        exp_u = np.exp(utilities)
        choice_probs = exp_u / exp_u.sum(axis=1, keepdims=True)
    shares = np.mean(choice_probs[:, :J], axis=0)
    return shares

def approximate_share_derivatives(prices, market_data, v_draws):
    """Approximate ∂s_j/∂p_k using simulation"""
    J, n_draws = len(market_data), v_draws.shape[0]
    x, xi = market_data['x'].values, market_data['xi'].values
    sat, wired = market_data['satellite'].values, market_data['wired'].values
    utilities = (beta1 * x + xi + v_draws[:, 0:1] * sat + v_draws[:, 1:2] * wired + alpha * prices)
    utilities = np.column_stack([utilities, np.zeros(n_draws)])
    exp_u = np.exp(utilities)
    choice_probs = exp_u / exp_u.sum(axis=1, keepdims=True)
    inside_shares_draws = choice_probs[:, :J]
    derivatives = np.zeros((J, J))
    for j in range(J):
        for k in range(J):
            if j == k:
                deriv_jk = inside_shares_draws[:, j] * (1 - inside_shares_draws[:, j])
            else:
                deriv_jk = -inside_shares_draws[:, j] * inside_shares_draws[:, k]
            derivatives[j, k] = alpha * np.mean(deriv_jk)  # Include α factor
    return derivatives

The derivative $\partial s_{jt}/\partial p_{kt}$ is approximated using Monte Carlo simulation. For each simulation draw $r = 1, \dots, R$ of the random coefficients $(\beta_i^{(2)}, \beta_i^{(3)})$, compute the conditional choice probability $P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)})$ and its derivative with respect to prices.

The derivative of the conditional choice probability follows from the multinomial logit formula:

$$\frac{\partial P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)})}{\partial p_{kt}} = \alpha P(j|\beta_i) \left( \delta_{jk} - P(k|\beta_i) \right)$$

where $\delta_{jk} = 1$ if $j = k` and 0 otherwise.

Then, the market share derivative is approximated as:

$$\frac{\partial s_{jt}}{\partial p_{kt}} \approx \frac{1}{R} \sum_{r=1}^R \frac{\partial P(\text{choose } j | \beta_i^{(2,r)}, \beta_i^{(3,r)})}{\partial p_{kt}}$$

Regarding "jittering": When solving for equilibrium prices iteratively, redrawing simulation draws in each iteration introduces random noise that can prevent convergence. To avoid this, pre-draw a fixed set of simulation draws for each market and reuse them throughout the solution process.

In [ ]:
# Pre-draw simulation draws (to avoid jittering)
n_draws = 100
all_v_draws = [np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws) for _ in range(T)]

(iv) Experiment to see how many simulation draws you need to get precise approximations and check this again at the equilibrium shares and prices you obtained below.

In [ ]:
# Marginal costs: ln(mc_jt) = γ₀ + w_jt γ₁ + ω_jt/8
product_data['mc'] = np.exp(gamma0 + gamma1 * product_data['w'] + product_data['omega'] / 8)

# Test on market 0
market_data = product_data[product_data['market_id'] == 0]
prices_initial = market_data['mc'].values * 1.2  # Initial prices 20% above MC
draw_counts = [50, 100, 200, 500, 1000, 2000, 5000]
print("Testing derivative approximation convergence at initial prices:")
print("Draws\t| Derivative Std Dev\t| Time (s)")
print("-" * 40)
initial_stds = []
for n_draws in draw_counts:
    start_time = time.time()

    # Compute derivatives at initial prices (5 repetitions for stability)
    deriv_initial_list = []
    for rep in range(5):
        v_draws = np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws)
        deriv = approximate_share_derivatives(prices_initial, market_data, v_draws)
        deriv_initial_list.append(deriv)

    deriv_initial_avg = np.mean(deriv_initial_list, axis=0)
    deriv_initial_std = np.std(deriv_initial_list, axis=0)
    initial_stds.append(deriv_initial_std.mean())

    computation_time = time.time() - start_time

    print(f"{n_draws:6d}\t| {deriv_initial_std.mean():.2e}\t\t| {computation_time:.2f}")

# Determine stabilization point for initial prices
initial_stds = np.array(initial_stds)
threshold = 0.001
stable_idx = None
for i in range(1, len(initial_stds)):
    if initial_stds[i] < threshold and abs(initial_stds[i] - initial_stds[i-1]) / initial_stds[i-1] < 0.5:
        stable_idx = i
        break

if stable_idx is not None:
    stable_draws = draw_counts[stable_idx]
    print(f"\nCONCLUSION: At initial prices 20% above MC, derivatives stabilize with {stable_draws} simulation draws.")
else:
    print(f"\nCONCLUSION: At initial prices 20% above MC, derivatives show decreasing variance, with best stability at {draw_counts[np.argmin(initial_stds)]} draws.")

(b) The FOC for firm $j$'s profit maximization problem in market $t$ is

\begin{align}
(p_{jt} - mc_{jt}) \frac{\partial s_{jt}}{\partial p_{jt}} + s_{jt} &= 0 \notag \\
\implies p_{jt} - mc_{jt} &= -\left( \frac{\partial s_{jt}}{\partial p_{jt}} \right)^{-1} s_{jt}
\end{align}

In [ ]:

print(f"MC range: {product_data['mc'].min():.3f} to {product_data['mc'].max():.3f}")
print(f"MC mean: {product_data['mc'].mean():.3f}, median: {product_data['mc'].median():.3f}")
print("FOC: (p_jt - mc_jt) * ∂s_jt/∂p_jt + s_jt = 0")
print("Rearranged: p_jt - mc_jt = - (∂s_jt/∂p_jt)⁻¹ * s_jt")

(c) Substituting in your approximation of each $\partial s_{jt}/\partial p_{jt}$, solve the system of equations above ($J$ equations per market) for the equilibrium prices in each market.

**i.** First do this using Matlab's "fsolve" operator. Check the exit flag from fsolve to be sure whether you found a solution for each market.

In [ ]:
def solve_prices_direct(market_data, mc_market, v_draws):
    """Direct nonlinear solver (equivalent to fsolve)"""
    def residual(prices):
        shares = compute_market_shares(prices, market_data, v_draws)
        derivatives = approximate_share_derivatives(prices, market_data, v_draws)
        markup = -np.linalg.inv(derivatives) @ shares
        return prices - mc_market - markup

    sol = opt.root(residual, mc_market, method='lm')
    return sol.x, sol.success

# Solve using direct method
equilibrium_prices_direct = []
success_flags_direct = []

for t in range(T):
    market_data = product_data[product_data['market_id'] == t]
    mc_market = market_data['mc'].values
    v_draws = all_v_draws[t] # Pre-drawn simulation draws (to avoid jittering)

    prices_direct, success = solve_prices_direct(market_data, mc_market, v_draws)
    equilibrium_prices_direct.append(prices_direct)
    success_flags_direct.append(success)

equilibrium_prices_direct = np.array(equilibrium_prices_direct)
print("Question 2(c)i completed:")
print(f"Direct method (fsolve equivalent): {sum(success_flags_direct)}/{T} markets solved")
print(f"Failed markets: {T - sum(success_flags_direct)}")
print(f"Price range: {equilibrium_prices_direct.min():.3f} to {equilibrium_prices_direct.max():.3f}")
print(f"Price mean: {equilibrium_prices_direct.mean():.3f}, std: {equilibrium_prices_direct.std():.3f}")

# Quick validation on first market
if sum(success_flags_direct) > 0:
    market_0 = product_data[product_data['market_id'] == 0]
    prices_0 = equilibrium_prices_direct[0]
    mc_0 = market_0['mc'].values
    v_draws_0 = all_v_draws[0] # Pre-drawn simulation draws (to avoid jittering)

    shares_0 = compute_market_shares(prices_0, market_0, v_draws_0)
    deriv_0 = approximate_share_derivatives(prices_0, market_0, v_draws_0)
    markup_0 = -np.linalg.inv(deriv_0) @ shares_0
    foc_residual_0 = prices_0 - mc_0 - markup_0

    print(f"Market 1 FOC check: max residual = {np.max(np.abs(foc_residual_0)):.2e}")
    print(f"Market 1 shares: {shares_0} (sum = {shares_0.sum():.3f})")

ii. Do this again using the algorithm of Morrow and Skerlos (2011), discussed in section 3.6 of Conlon and Gortmaker (2019) (and in the pyBLP "problem simulation tutorial"). Use the numerical integration approach you used in step (a) to approximate the terms defined in equation (25) of Conlon and Gortmaker. If you get different results using this method, resolve this discrepancy either by correcting your code or explaining why your preferred method is the one to be trusted.

In [4]:
def solve_prices_morrow_skerlos(market_data, mc_market, v_draws, max_iter=100, tol=1e-6):
    """Morrow-Skerlos algorithm"""
    prices = mc_market.copy()

    for iteration in range(max_iter):
        utilities = (beta1 * market_data['x'].values + market_data['xi'].values +
                     v_draws[:, 0:1] * market_data['satellite'].values +
                     v_draws[:, 1:2] * market_data['wired'].values +
                     alpha * prices)

        utilities = np.column_stack([utilities, np.zeros(v_draws.shape[0])])
        exp_u = np.exp(utilities)
        choice_probs = exp_u / exp_u.sum(axis=1, keepdims=True)
        inside_shares_draws = choice_probs[:, :len(market_data)]
        shares = compute_market_shares(prices, market_data, v_draws)

        Lambda = np.diag(alpha * shares)
        Gamma = alpha * (inside_shares_draws.T @ inside_shares_draws) / v_draws.shape[0]

        diff = prices - mc_market
        zeta = np.linalg.inv(Lambda) @ Gamma.T @ diff - np.linalg.inv(Lambda) @ shares
        prices_new = mc_market + zeta

        foc_residual = Lambda @ (prices - mc_market - zeta)
        if np.max(np.abs(foc_residual)) < tol:
            break

        prices = prices_new

    return prices, iteration + 1

# Solve using Morrow-Skerlos method
equilibrium_prices_ms = []
iterations_ms = []

for t in range(T):
    market_data = product_data[product_data['market_id'] == t]
    mc_market = market_data['mc'].values
    v_draws = all_v_draws[t]

    prices_ms, iters = solve_prices_morrow_skerlos(market_data, mc_market, v_draws)
    equilibrium_prices_ms.append(prices_ms)
    iterations_ms.append(iters)

equilibrium_prices_ms = np.array(equilibrium_prices_ms)
print("Question 2(c)ii completed:")
print(f"Morrow-Skerlos method: All {T} markets solved")
print(f"Average iterations: {np.mean(iterations_ms):.1f}")
print(f"Max iterations: {np.max(iterations_ms)}")
print(f"Price range: {equilibrium_prices_ms.min():.3f} to {equilibrium_prices_ms.max():.3f}")
print(f"Price mean: {equilibrium_prices_ms.mean():.3f}, std: {equilibrium_prices_ms.std():.3f}")

# Use Morrow-Skerlos prices
product_data['prices'] = equilibrium_prices_ms.flatten()

# Compare direct vs Morrow-Skerlos
price_diff = np.abs(equilibrium_prices_direct - equilibrium_prices_ms)
print(f"Max price difference between methods: {price_diff.max():.2e}")
print(f"Mean price difference: {price_diff.mean():.2e}")

# Final validation on first market
market_0 = product_data[product_data['market_id'] == 0]
prices_0 = market_0['prices'].values
mc_0 = market_0['mc'].values
v_draws_0 = all_v_draws[0]

shares_0 = compute_market_shares(prices_0, market_0, v_draws_0)
deriv_0 = approximate_share_derivatives(prices_0, market_0, v_draws_0)
markup_0 = -np.linalg.inv(deriv_0) @ shares_0
foc_residual_0 = prices_0 - mc_0 - markup_0

print(f"Equilibrium FOC check (market 1): max residual = {np.max(np.abs(foc_residual_0)):.2e}")
print(f"Equilibrium shares (market 1): {shares_0} (sum = {shares_0.sum():.3f})")

KeyError: 'mc'

In [ ]:
# Compare derivative approximation convergence at initial vs equilibrium prices

# Test on market 0
market_data = product_data[product_data['market_id'] == 0]
prices_initial = market_data['mc'].values * 1.2  # Initial prices 20% above MC
prices_equilibrium = market_data['prices'].values

draw_counts = [50, 100, 200, 500, 1000, 2000, 5000]

print("Comparing derivative approximation convergence at initial vs equilibrium prices:")
print("Draws\t| Initial Price Std Dev\t| Equilibrium Price Std Dev\t| Ratio (Eq/Init)")
print("-" * 80)

initial_stds = []
eq_stds = []
for n_draws in draw_counts:

    # Compute derivatives at initial prices
    deriv_initial_list = []
    for rep in range(5):
        v_draws = np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws)
        deriv = approximate_share_derivatives(prices_initial, market_data, v_draws)
        deriv_initial_list.append(deriv)

    deriv_initial_std = np.std(deriv_initial_list, axis=0)
    initial_stds.append(deriv_initial_std.mean())

    # Compute derivatives at equilibrium prices
    deriv_eq_list = []
    for rep in range(5):
        v_draws = np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws)
        deriv = approximate_share_derivatives(prices_equilibrium, market_data, v_draws)
        deriv_eq_list.append(deriv)

    deriv_eq_std = np.std(deriv_eq_list, axis=0)
    eq_stds.append(deriv_eq_std.mean())

    ratio = deriv_eq_std.mean() / deriv_initial_std.mean() if deriv_initial_std.mean() > 0 else float('inf')

    print(f"{n_draws:6d}\t| {deriv_initial_std.mean():.2e}\t\t| {deriv_eq_std.mean():.2e}\t\t\t| {ratio:.2f}")


print(f"Average ratio of std dev (equilibrium/initial): {np.mean([eq_stds[i]/initial_stds[i] for i in range(len(initial_stds)) if initial_stds[i] > 0]):.2f}")

avg_ratio = np.mean([eq_stds[i]/initial_stds[i] for i in range(len(initial_stds)) if initial_stds[i] > 0])
reduction_pct = (1 - avg_ratio) * 100
print(f"\nCONCLUSION: Derivatives at equilibrium prices are, on average, {reduction_pct:.1f}% less variable than at initial prices.")

### 3. 
Calculate "observed" market shares for your fake data set using your parameters, your draws of $x$, $w$, $\xi$, $\omega$, and your equilibrium prices.

In [ ]:
observed_shares = []
for t in range(T):
    market_data = product_data[product_data['market_id'] == t]
    prices_market = market_data['prices'].values

    shares_market = compute_market_shares(prices_market, market_data, n_draws=10000)
    observed_shares.extend(shares_market)

product_data['shares'] = observed_shares

print(f"Share range: {product_data['shares'].min():.3f} to {product_data['shares'].max():.3f}")
print(f"Share mean: {product_data['shares'].mean():.3f}, std: {product_data['shares'].std():.3f}")

# Validation: Check market share sums
market_share_sums = product_data.groupby('market_id')['shares'].sum()
print(f"Market share sums (should be < 1):")
print(f"Average: {market_share_sums.mean():.3f}")
print(f"Min: {market_share_sums.min():.3f}, Max: {market_share_sums.max():.3f}")
print(f"Outside shares: {1 - market_share_sums.mean():.3f} (average)")

# Check by product type
satellite_shares = product_data[product_data['satellite'] == 1]['shares'].mean()
wired_shares = product_data[product_data['wired'] == 1]['shares'].mean()
print(f"Average satellite product share: {satellite_shares:.3f}")
print(f"Average wired product share: {wired_shares:.3f}")

### 4. 

Below you'll be using $x$ and $w$ as instruments in the demand estimation. Check whether these appear to be good instruments in your fake data using some regressions of prices and market shares on the exogenous variables (or some function of them; see the related discussion in the coding tips). If you believe the instruments are not providing enough variation, modify the parameter choices above until you are satisfied. Report your final choice of parameters and the results you rely on to conclude that the instruments seem good enough.

In [ ]:
# Prepare data for regressions with extended instrument set
# Create quadratic and interaction columns first
product_data['x**2'] = product_data['x'] ** 2
product_data['w**2'] = product_data['w'] ** 2
product_data['x*w'] = product_data['x'] * product_data['w']
X_instruments = sm.add_constant(product_data[['x', 'w', 'x**2', 'w**2', 'x*w']])

# Regression 1: Prices on extended instruments (Relevance check)
price_model = sm.OLS(product_data['prices'], X_instruments).fit()
print("Regression: Prices ~ x + w + x² + w² + x*w (Relevance Check)")
print(f"R-squared: {price_model.rsquared:.3f}")
print(f"F-statistic: {price_model.fvalue:.2f} (p-value: {price_model.f_pvalue:.2e})")
print()

# Regression 2: Market shares on extended instruments
share_model = sm.OLS(product_data['shares'], X_instruments).fit()
print("Regression: Shares ~ x + w + x² + w² + x*w")
print(f"R-squared: {share_model.rsquared:.3f}")
print(f"F-statistic: {share_model.fvalue:.2f} (p-value: {share_model.f_pvalue:.2e})")
print()

# Regression 3: Demand unobservable ξ on instruments (Exclusion check)
xi_model = sm.OLS(product_data['xi'], X_instruments).fit()
print("Regression: ξ ~ x + w + x² + w² + x*w (Exclusion Check)")
print(f"R-squared: {xi_model.rsquared:.3f}")
print(f"F-statistic: {xi_model.fvalue:.2f} (p-value: {xi_model.f_pvalue:.2e})")

# Assess instrument strength and validity
weak_instruments = (price_model.f_pvalue >= 0.01 and share_model.f_pvalue >= 0.01) or (price_model.rsquared < 0.05 and share_model.rsquared < 0.05)
excluded_instruments = xi_model.f_pvalue < 0.01  # Should be false for valid exclusion
print()
print("FINAL PARAMETER CHOICE:")
if weak_instruments or excluded_instruments:
    print("Parameters need adjustment - instruments are weak or invalid.")
else:
    print(f"Demand: α = {alpha}, β^(1) = {beta1}, β_i^(2) ~ N({beta2}, {sigma_satellite}²), β_i^(3) ~ N({beta3}, {sigma_wired}²)")
    print(f"Supply: γ^(0) = {gamma0}, γ^(1) = {gamma1}")
    print("These parameters generate data with valid instruments and are retained as final.")

## 4 Estimate Some Mis-specified Models

### 5. Estimate the plain multinomial logit model of demand by OLS (ignoring the endogeneity of prices).

For the plain multinomial logit model, the utility is:

$$u_{ijt} = \beta^{(1)} x_{jt} + \beta^{(2)} satellite_{jt} + \beta^{(3)} wired_{jt} + \alpha p_{jt} + \xi_{jt} + \epsilon_{ijt}$$

This implies the log-odds ratio:

$$\ln\left(\frac{s_{jt}}{s_{0t}}\right) = \delta_{jt} = \beta^{(1)} x_{jt} + \beta^{(2)} satellite_{jt} + \beta^{(3)} wired_{jt} + \alpha p_{jt} + \xi_{jt}$$

We can estimate this by OLS, regressing the logit-transformed shares on the observed product characteristics.

In [ ]:
# Compute outside shares for each market
product_data['outside_share'] = 1 - product_data.groupby('market_id')['shares'].transform('sum')

# Compute logit delta: ln(s_jt / s_0t)
product_data['logit_delta'] = np.log(product_data['shares'] / product_data['outside_share'])

# OLS using matrix algebra (no intercept)
y = product_data['logit_delta'].values
X = product_data[['x', 'satellite', 'wired', 'prices']].values

# Compute OLS estimates: beta_hat = (X^T X)^(-1) X^T y
beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y

# Compute residuals and clustered standard errors
y_hat = X @ beta_hat
residuals = y - y_hat
n, k = X.shape

# Clustered covariance matrix by market
clusters = product_data['market_id'].values
unique_clusters = np.unique(clusters)
V = np.zeros((k, k))
for c in unique_clusters:
    mask = clusters == c
    X_c = X[mask]
    e_c = residuals[mask]
    V += X_c.T @ np.outer(e_c, e_c) @ X_c
cov_matrix_ols = np.linalg.inv(X.T @ X) @ V @ np.linalg.inv(X.T @ X)
se_ols = np.sqrt(np.diag(cov_matrix_ols))

# t-statistics and p-values
t_stats = beta_hat / se_ols
p_values = 2 * (1 - stats.norm.cdf(np.abs(t_stats)))

print("OLS Regression: ln(s_jt/s_0t) ~ x + satellite + wired + prices (no intercept)")
print("-" * 70)
param_names = ['x', 'satellite', 'wired', 'prices']
for i, param in enumerate(param_names):
    print(f"{param:12s}: {beta_hat[i]:8.3f} (SE: {se_ols[i]:.3f}, t: {t_stats[i]:6.2f}, p: {p_values[i]:.3f})")

### 6. 
Re-estimate the multinomial logit model of demand by two-stage
least squares, instrumenting for prices with the exogenous demand shifters $%
x $ and excluded cost shifters w. Discuss how the results differ from those
obtained by OLS.

In [ ]:
# Additional instruments
if 'w**2' not in product_data.columns:
    product_data['w**2'] = product_data['w'] ** 2
if 'x**2' not in product_data.columns:
    product_data['x**2'] = product_data['x'] ** 2
if 'x*w' not in product_data.columns:
    product_data['x*w'] = product_data['x'] * product_data['w']
if 'sum_x_competitors' not in product_data.columns:
    product_data['sum_x_competitors'] = product_data.groupby('market_id')['x'].transform('sum') - product_data['x']
if 'sum_w_competitors' not in product_data.columns:
    product_data['sum_w_competitors'] = product_data.groupby('market_id')['w'].transform('sum') - product_data['w']

# First stage: 
Z = product_data[['x', 'satellite', 'wired', 'w', 'x**2', 'w**2', 'x*w', 'sum_x_competitors', 'sum_w_competitors']].values  

# First stage OLS: prices ~ x + w + x² + w²
Z_with_const = np.column_stack([np.ones(len(Z)), Z])
pi_hat = np.linalg.inv(Z_with_const.T @ Z_with_const) @ Z_with_const.T @ product_data['prices'].values
prices_hat = Z_with_const @ pi_hat

# Second stage: Regress logit_delta on x + satellite + wired + predicted_prices
y = product_data['logit_delta'].values
X_second = np.column_stack([
    product_data['x'].values,
    product_data['satellite'].values,
    product_data['wired'].values,
    prices_hat  # Use predicted prices from first stage
])

# 2SLS estimates: beta_hat_iv = (X_second^T X_second)^(-1) X_second^T y
beta_hat_iv = np.linalg.inv(X_second.T @ X_second) @ X_second.T @ y

# Compute GMM standard errors (to match PyBLP)
residuals_iv = y - X_second @ beta_hat_iv

# Clustered covariance of moments S
clusters = product_data['market_id'].values
unique_clusters = np.unique(clusters)
S = np.zeros((Z.shape[1], Z.shape[1]))
for c in unique_clusters:
    mask = clusters == c
    Z_c = Z[mask]
    e_c = residuals_iv[mask]
    g_c = Z_c.T @ e_c
    S += np.outer(g_c, g_c)
S = S / len(product_data)  # Divide by N

# Optimal weighting matrix W = S^{-1}
W_opt = np.linalg.inv(S)

# Jacobian G = Z' X_second
G = Z.T @ X_second

# GMM covariance: (G' W G)^{-1} G' W S W G (G' W G)^{-1}
GWG_inv = np.linalg.inv(G.T @ W_opt @ G)
cov_matrix_iv = GWG_inv @ G.T @ W_opt @ S @ W_opt @ G @ GWG_inv
# cov_matrix_iv = cov_matrix_iv / len(product_data)  # Remove this
se_iv = np.sqrt(np.diag(cov_matrix_iv)) * np.sqrt(len(product_data)) 
t_stats_iv = beta_hat_iv / se_iv
p_values_iv = 2 * (1 - stats.norm.cdf(np.abs(t_stats_iv)))

print("2SLS IV Regression: ln(s_jt/s_0t) ~ x + satellite + wired + prices_hat (no intercept)")
print("First stage instruments: x, w, x², w², x*w, sum_x_competitors, sum_w_competitors")
print("-" * 80)
param_names = ['x', 'satellite', 'wired', 'prices']
for i, param in enumerate(param_names):
    print(f"{param:12s}: {beta_hat_iv[i]:8.3f} (SE: {se_iv[i]:.3f}, t: {t_stats_iv[i]:6.2f}, p: {p_values_iv[i]:.3f})")

### 7. Nested Logit Model Estimation

Now estimate a nested logit model by two-stage least squares, treating "satellite" and "wired" as the two nests for the inside goods. You will probably want to review the discussion of the nested logit in Berry (1994). Note that Berry focuses on the special case in which all the "nesting parameters" are the same; you should allow a different nesting parameter for each nest.

In Berry's notation, this means letting the parameter σ become σ_{g(j)}, where g(j) indicates the group (satellite or wired) to which each inside good j belongs.

Without reference to the results, explain the way(s) that this model is misspecified. (Hint: students tend to get this question wrong; recall that I suggested you review Berry 94).

In [ ]:
# Compact nested logit estimation using manual GMM
print("Question 7: Nested Logit Model Estimation")
print("=" * 50)

# Use all markets for estimation
print("Nested logit estimation with GMM...")

# Add mean x per group to instruments
product_data['nesting_ids'] = product_data['satellite'].map({1: 'satellite', 0: 'wired'})
product_data['group_size'] = product_data.groupby(['market_id', 'nesting_ids'])['w'].transform('mean')
product_data['x_other_in_nest'] = product_data.groupby(['market_id', 'nesting_ids'])['x'].transform('sum') - product_data['x']
product_data['w_other_in_nest'] = product_data.groupby(['market_id', 'nesting_ids'])['w'].transform('sum') - product_data['w']

# Prepare instruments
Z = product_data[['x', 'satellite', 'wired', 'w', 'x**2', 'w**2', 'x*w', 'group_size', 'sum_x_competitors', 'sum_w_competitors', 'x_other_in_nest', 'w_other_in_nest']].values

# Pre-draw simulation draws (to avoid jittering)
n_draws = 100
all_v_draws = [np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws) for _ in range(T)]

def compute_market_shares_nested(delta, market_data, v_draws, beta, rho_sat, rho_wired):
    beta_x, beta_sat, beta_wired, beta_price = beta
    J = len(market_data)
    s_j = np.zeros((n_draws, J))
    for d in range(n_draws):
        utilities = np.zeros(J)
        for j in range(J):
            row = market_data.iloc[j]
            utilities[j] = delta[j] + beta_x * row['x'] + beta_sat * row['satellite'] + beta_wired * row['wired'] + beta_price * row['prices'] + v_draws[d, 0] * row['satellite'] + v_draws[d, 1] * row['wired']
        # Compute nested shares
        sat_mask = market_data['satellite'] == 1
        wired_mask = market_data['satellite'] == 0
        sat_indices = market_data[sat_mask].index
        wired_indices = market_data[wired_mask].index
        nests = [('sat', rho_sat, sat_indices), ('wired', rho_wired, wired_indices)]
        nest_shares = {}
        for nest_name, rho_h, indices in nests:
            if len(indices) == 0:
                continue
            u_h = utilities[[market_data.index.get_loc(idx) for idx in indices]]
            if rho_h == 0:
                scaled_u = u_h
            else:
                scaled_u = u_h / (1 - rho_h)
            max_scaled = np.max(scaled_u)
            exp_scaled = np.exp(scaled_u - max_scaled)
            sum_exp = np.sum(exp_scaled)
            s_jh = exp_scaled / sum_exp
            log_sum_h = np.log(sum_exp) + max_scaled
            nest_u_h = rho_h * log_sum_h
            nest_shares[nest_name] = (nest_u_h, s_jh, indices)
        # Compute nest shares
        if len(nest_shares) == 0:
            continue
        nest_us = [nest_shares[name][0] for name in nest_shares]
        max_nest_u = np.max(nest_us)
        exp_nest_scaled = np.exp(np.array(nest_us) - max_nest_u)
        exp_term = np.exp(np.clip(-max_nest_u, -500, 500))
        denom = np.sum(exp_nest_scaled) + exp_term
        s_h = exp_nest_scaled / denom
        # Assign s_j
        for i, (nest_name, rho_h, indices) in enumerate(nests):
            if nest_name in nest_shares:
                s_jh = nest_shares[nest_name][1]
                for k, idx in enumerate(indices):
                    j = market_data.index.get_loc(idx)
                    s_j[d, j] = s_h[i] * s_jh[k]
    return s_j.mean(axis=0)

def solve_delta_market(market_data, beta, rho_sat, rho_wired, v_draws):
    beta_x, beta_sat, beta_wired, beta_price = beta
    s_j = np.maximum(market_data['shares'].values, 1e-15)
    s_0 = 1 - s_j.sum()
    
    sat_products = market_data[market_data['satellite'] == 1]
    wired_products = market_data[market_data['wired'] == 1]
    s_sat = sat_products['shares'].sum() if len(sat_products) > 0 else 0
    s_wired = wired_products['shares'].sum() if len(wired_products) > 0 else 0
    
    delta = np.zeros(len(s_j))
    for i, (_, row) in enumerate(market_data.iterrows()):
        s_j_val = row['shares']
        x_val = row['x']
        sat_val = row['satellite']
        wired_val = row['wired']
        price_val = row['prices']
        observed_part = beta_x * x_val + beta_sat * sat_val + beta_wired * wired_val + beta_price * price_val
        if row['satellite'] == 1:
            rho = rho_sat
            s_h = s_sat
        else:
            rho = rho_wired
            s_h = s_wired
        if s_h > 0 and s_j_val > 0 and s_0 > 0:
            delta[i] = (1 - rho) * (np.log(s_j_val) - observed_part) + rho * np.log(s_h) - np.log(s_0)
        else:
            delta[i] = 0
    
    rho_array = np.where(market_data['satellite'] == 1, rho_sat, rho_wired)
    dampener = 1 - rho_array
    for iteration in range(50):
        shares_model = compute_market_shares_nested(delta, market_data, v_draws, beta, rho_sat, rho_wired)
        shares_model = np.maximum(shares_model, 1e-15)
        delta_new = delta + dampener * (np.log(s_j) - np.log(shares_model))
        if np.max(np.abs(delta_new - delta)) < 1e-8:
            break
        delta = delta_new
    return delta

def gmm_objective(params, product_data, Z, W):
    beta_x, beta_sat, beta_wired, beta_price, rho_sat, rho_wired = params
    all_delta = []
    all_X = []
    all_Z_local = []
    for t in product_data['market_id'].unique():
        market_data = product_data[product_data['market_id'] == t]
        v_draws = all_v_draws[t]
        delta = solve_delta_market(market_data, [beta_x, beta_sat, beta_wired, beta_price], rho_sat, rho_wired, v_draws)
        for idx in market_data.index:
            row = market_data.loc[idx]
            X_j = np.array([row['x'], row['satellite'], row['wired'], row['prices']])
            Z_j = Z[product_data.index.get_loc(idx)]
            all_delta.append(delta[market_data.index.get_loc(idx)])
            all_X.append(X_j)
            all_Z_local.append(Z_j)
    all_delta = np.array(all_delta)
    all_X = np.array(all_X)
    all_Z = np.array(all_Z_local)
    # Solve beta
    XZWZX = all_X.T @ all_Z @ W @ all_Z.T @ all_X
    XZWZd = all_X.T @ all_Z @ W @ all_Z.T @ all_delta
    beta_hat = np.linalg.solve(XZWZX, XZWZd)
    xi = all_delta - all_X @ beta_hat
    g = all_Z.T @ xi
    objective = (1 / len(product_data)) * g.T @ W @ g
    return objective, xi

# Initial guesses
initial_params = [beta_hat_iv[0], beta_hat_iv[1], beta_hat_iv[2], beta_hat_iv[3], 0.5, 0.5]  # beta_x, beta_sat, beta_wired, beta_price, rho_sat, rho_wired
bounds = [(-np.inf, np.inf), (-np.inf, np.inf), (-np.inf, np.inf), (-np.inf, np.inf), (0, 1), (0, 1)]

# Step 1: 1-step GMM with W = (Z'Z)^{-1}
W1 = np.linalg.inv(Z.T @ Z)
result1 = opt.minimize(lambda params: gmm_objective(params, product_data, Z, W1)[0], initial_params, bounds=bounds, method='L-BFGS-B')
params1 = result1.x

# Compute S1 from first step residuals
_, residuals1 = gmm_objective(params1, product_data, Z, W1)
S1 = np.zeros((Z.shape[1], Z.shape[1]))
for c in unique_clusters:
    mask = clusters == c
    Z_c = Z[mask]
    e_c = residuals1[mask]
    g_c = Z_c.T @ e_c
    S1 += np.outer(g_c, g_c)
S1 /= len(product_data)
W2 = np.linalg.inv(S1)

# Step 3: 2-step GMM with W2
result2 = opt.minimize(lambda params: gmm_objective(params, product_data, Z, W2)[0], params1, bounds=bounds, method='L-BFGS-B')
beta_x_opt, beta_sat_opt, beta_wired_opt, beta_price_opt, rho_sat_opt, rho_wired_opt = result2.x
beta_hat_nested = np.array([beta_x_opt, beta_sat_opt, beta_wired_opt, beta_price_opt])

print(f"2-step GMM converged: {result2.success}")
print(f"Optimal parameters:")
print(f"β_x = {beta_x_opt:.3f}, β_satellite = {beta_sat_opt:.3f}, β_wired = {beta_wired_opt:.3f}, β_price = {beta_price_opt:.3f}")
print(f"ρ_satellite = {rho_sat_opt:.3f}, ρ_wired = {rho_wired_opt:.3f}")

# Compute GMM standard errors for nested logit
_, residuals_final = gmm_objective(result2.x, product_data, Z, W2)

# Compute Jacobian G = Z' @ d residual / d theta
d_residual_d_theta = np.column_stack([
    -product_data['x'].values,
    -product_data['satellite'].values,
    -product_data['wired'].values,
    -product_data['prices'].values,
    np.zeros(len(product_data)),  # for rho_sat
    np.zeros(len(product_data))   # for rho_wired
])

for t in product_data['market_id'].unique():
    market_data = product_data[product_data['market_id'] == t]
    sat_products = market_data[market_data['satellite'] == 1]
    wired_products = market_data[market_data['wired'] == 1]
    s_sat = sat_products['shares'].sum() if len(sat_products) > 0 else 0
    s_wired = wired_products['shares'].sum() if len(wired_products) > 0 else 0
    for idx in market_data.index:
        s_j = product_data.loc[idx, 'shares']
        if product_data.loc[idx, 'satellite'] == 1:
            if s_sat > 0 and s_j > 0:
                d_residual_d_theta[idx, 4] = -np.log(s_j) + np.log(s_sat)
        else:
            if s_wired > 0 and s_j > 0:
                d_residual_d_theta[idx, 5] = -np.log(s_j) + np.log(s_wired)

G = Z.T @ d_residual_d_theta

# Compute S
clusters = product_data['market_id'].values
unique_clusters = np.unique(clusters)
S = np.zeros((Z.shape[1], Z.shape[1]))
for c in unique_clusters:
    mask = clusters == c
    Z_c = Z[mask]
    e_c = residuals_final[mask]
    g_c = Z_c.T @ e_c
    S += np.outer(g_c, g_c)
S = S / len(product_data)

W_opt = np.linalg.inv(S)
GWG_inv = np.linalg.inv(G.T @ W_opt @ G)
cov_matrix_nested = GWG_inv @ G.T @ W_opt @ S @ W_opt @ G @ GWG_inv
se_nested = np.sqrt(np.diag(cov_matrix_nested))

# t-statistics and p-values
t_stats = beta_hat_nested / se_nested[:4]
p_values = 2 * (1 - stats.norm.cdf(np.abs(t_stats)))
t_stats_rho = result2.x[4:] / se_nested[4:]
p_values_rho = 2 * (1 - stats.norm.cdf(np.abs(t_stats_rho)))

print("\nNested Logit 2-step GMM: δ_jt ~ x + satellite + wired + prices (no intercept)")
print("Instruments: x, w, x², w², x*w, mean_w_group, sum_x_competitors, sum_w_competitors, x_other_in_nest, w_other_in_nest")
print("-" * 80)
param_names = ['x', 'satellite', 'wired', 'prices']
for i, param in enumerate(param_names):
    print(f"{param:12s}: {beta_hat_nested[i]:8.3f} (SE: {se_nested[i]:.3f}, t: {t_stats[i]:6.2f}, p: {p_values[i]:.3f})")
print(f"ρ_satellite  : {rho_sat_opt:.3f} (SE: {se_nested[4]:.3f}, t: {t_stats_rho[0]:6.2f}, p: {p_values_rho[0]:.3f})")
print(f"ρ_wired      : {rho_wired_opt:.3f} (SE: {se_nested[5]:.3f}, t: {t_stats_rho[1]:6.2f}, p: {p_values_rho[1]:.3f})")

print(f"Adjusted price coefficient α/(1-ρ_satellite): {beta_price_opt / (1 - rho_sat_opt)}")
print(f"Adjusted price coefficient α/(1-ρ_wired): {beta_price_opt / (1 - rho_wired_opt)}")

## PyBLP Implementations

Now let's implement the same models using PyBLP for comparison.

In [ ]:
import pyblp
pyblp.options.digits = 3
pyblp.options.verbose = False
pyblp.options.collinear_atol = 0
pyblp.options.collinear_rtol = 0
pyblp.options.singular_tol = 0
pd.options.display.precision = 3
pd.options.display.max_columns = 50


In [1]:
# Prepare data for PyBLP
pyblp_data = product_data.copy()
pyblp_data = pyblp_data.rename(columns={'market_id': 'market_ids'})

# Ensure necessary columns are present
pyblp_data['sum_x_competitors'] = pyblp_data.groupby('market_ids')['x'].transform('sum') - pyblp_data['x']
pyblp_data['sum_w_competitors'] = pyblp_data.groupby('market_ids')['w'].transform('sum') - pyblp_data['w']

print("Data prepared for PyBLP:")
print(f"Markets: {pyblp_data['market_ids'].nunique()}")
print(f"Products per market: {pyblp_data.groupby('market_ids').size().iloc[0]}")


NameError: name 'product_data' is not defined

In [ ]:
# PyBLP OLS for logit (treat prices as exogenous)
ols_data = pyblp_data.drop(columns=[c for c in pyblp_data.columns if 'demand_instruments' in c or c == 'nesting_ids'])
ols_formulation = pyblp.Formulation('0 + prices + x + satellite + wired')
ols_problem = pyblp.Problem(ols_formulation, ols_data)
logit_ols_results = ols_problem.solve(method='1s')
print("PyBLP Plain Logit OLS Results:")
print(logit_ols_results)

### Plain Logit IV

In [ ]:

# Add demand instruments
iv_data = pyblp_data.drop(columns=['nesting_ids'])
iv_data['demand_instruments0'] = iv_data['w']
iv_data['demand_instruments1'] = iv_data['sum_x_competitors']
iv_data['demand_instruments2'] = iv_data['sum_w_competitors']
iv_formulation = pyblp.Formulation('0 + prices + x + satellite + wired')
iv_problem = pyblp.Problem(iv_formulation, iv_data)
logit_results = iv_problem.solve()
print("PyBLP Plain Logit IV Results:")
print(logit_results)

### Nested Logit (PyBLP)

For nested logit, we need to add nesting_ids and an additional instrument for the within-group share.

In [ ]:
# Prepare data for nested logit
nested_data = pyblp_data.copy()
nested_data['nesting_ids'] = nested_data['satellite'].map({1: 'satellite', 0: 'wired'})

# Ensure sum_x_competitors and sum_w_competitors are present
nested_data['sum_x_competitors'] = nested_data.groupby('market_ids')['x'].transform('sum') - nested_data['x']
nested_data['sum_w_competitors'] = nested_data.groupby('market_ids')['w'].transform('sum') - nested_data['w']

# Compute mean x per group per market
nested_data['mean_x_group'] = nested_data.groupby(['market_ids', 'nesting_ids'])['x'].transform('mean')
nested_data['x_other_in_nest'] = nested_data.groupby(['market_ids', 'nesting_ids'])['x'].transform('sum') - nested_data['x']
nested_data['demand_instruments0'] = nested_data['x']
nested_data['demand_instruments1'] = nested_data['w']
nested_data['demand_instruments2'] = nested_data['satellite']
nested_data['demand_instruments3'] = nested_data['wired']
nested_data['demand_instruments4'] = nested_data['sum_x_competitors']
nested_data['demand_instruments5'] = nested_data['x_other_in_nest']

# Nested logit formulation
nl_formulation = pyblp.Formulation('0 + prices + x + satellite + wired')
nl_problem = pyblp.Problem(nl_formulation, nested_data)
rho_initial = [0.5, 0.5]  # Initial values for rho_sat and rho_wired
nl_results = nl_problem.solve(rho=rho_initial)

print("PyBLP Nested Logit Results:")
print(nl_results)
print(f"Price coefficient: {nl_results.beta[0]}")
print(f"Nesting parameters ρ: {nl_results.rho}")
print(f"Adjusted price coefficient α/(1-ρ_satellite): {nl_results.beta[0] / (1 - nl_results.rho[0])}")
print(f"Adjusted price coefficient α/(1-ρ_wired): {nl_results.beta[0] / (1 - nl_results.rho[1])}")

### Comparison with Manual Results

Compare the PyBLP results with the manual matrix algebra results.

In [ ]:
print("Comparison of Results:")
print("=" * 50)
print("Manual OLS α: {:.3f} (SE: {:.3f})".format(beta_hat[3], se_ols[3]))
print("Manual IV α: {:.3f} (SE: {:.3f})".format(beta_hat_iv[3], se_iv[3]))
print("Manual Nested α: {:.3f} (SE: {:.3f})".format(beta_price_opt, se_nested[3]))
print("Manual Nested ρ_satellite: {:.3f} (SE: {:.3f}), ρ_wired: {:.3f} (SE: {:.3f})".format(rho_sat_opt, se_nested[4], rho_wired_opt, se_nested[5]))
print("Manual Nested α/(1-ρ_satellite): {:.3f}".format(beta_price_opt / (1 - rho_sat_opt)))
print("Manual Nested α/(1-ρ_wired): {:.3f}".format(beta_price_opt / (1 - rho_wired_opt)))
print()
print("PyBLP Logit OLS α: {:.3f} (SE: {:.3f})".format(logit_ols_results.beta[0].item(), logit_ols_results.beta_se[0].item()))
print("PyBLP Logit IV α: {:.3f} (SE: {:.3f})".format(logit_results.beta[0].item(), logit_results.beta_se[0].item()))
print("PyBLP Nested α: {:.3f} (SE: {:.3f})".format(nl_results.beta[0].item(), nl_results.beta_se[0].item()))
print("PyBLP Nested ρ_satellite: {:.3f} (SE: {:.3f}), ρ_wired: {:.3f} (SE: {:.3f})".format(nl_results.rho[0].item(), nl_results.rho_se[0].item(), nl_results.rho[1].item(), nl_results.rho_se[1].item()))
print("PyBLP Nested α/(1-ρ_satellite): {:.3f}".format((nl_results.beta[0] / (1 - nl_results.rho[0])).item()))
print("PyBLP Nested α/(1-ρ_wired): {:.3f}".format((nl_results.beta[0] / (1 - nl_results.rho[1])).item()))
